In [50]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Load dataset
file_path = "/Users/abhiaremanda/BUCKSHACKATHON/CleanedData/AccountLevelFinal.csv" 
df = pd.read_csv(file_path)
df.head()

,Season,AccountNumber,SingleGameTickets,PartialPlanTickets,GroupTickets,AvgSpend,GamesAttended,FanSegment,DistanceToArena,BasketballPropensity,...,Tier_AB_Fraction,MostAttendedTier,WeekendFraction,TierA_Weekday_Fraction,TierB_Weekday_Fraction,TierC_Weekday_Fraction,TierD_Weekday_Fraction,TierAB_Weekday_Fraction,TierCD_Weekday_Fraction,GiveawayFraction
0,2023,1,0,0,0,467.0,0,F,12.0,872.0,...,0.0,Tier_D,0.0,0.0,0.0,0.0,1.0,0.0,0.5,1.0
1,2023,2,2,0,0,116.0,1,A,47.0,485.0,...,1.0,Tier_A,0.0,1.0,0.0,0.0,0.0,0.5,0.0,0.0
2,2023,3,3,0,0,107.0,1,B,6.0,896.0,...,0.0,Tier_D,0.0,0.0,0.0,0.0,1.0,0.0,0.5,0.0
3,2023,4,0,0,3,27.0,1,C,3.0,467.0,...,0.0,Tier_D,0.0,0.0,0.0,0.0,1.0,0.0,0.5,0.0
4,2023,5,0,0,2,14.0,1,A,4.0,582.0,...,0.0,Tier_D,0.0,0.0,0.0,0.0,1.0,0.0,0.5,0.0


In [51]:
if 'WeekendGameCount' not in df.columns:
    df['WeekendGameCount'] = df["TotalTicketsPurchased"] * df['WeekendFraction']

In [45]:
# Compute Total Weekend Games if not present
if 'WeekendGameCount' not in df.columns:
    df['WeekendGameCount'] = df['TotNumTicketsPurchased'] * df['WeekendFraction']

# Select only numeric columns for regression
numeric_columns = df.select_dtypes(include=['number']).columns.tolist()

# Remove the target variable from the feature list
features = [col for col in numeric_columns if col != 'WeekendGameCount']

X = df[features]
y = df['WeekendGameCount']


In [46]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

numeric_columns = df.select_dtypes(include=['number']).columns.tolist()

# Define X and y
X = df[numeric_columns]
X = X.drop(columns=['WeekendGameCount'], errors='ignore')  # Drop target variable
y = df['WeekendGameCount']

# Compute VIF after removing highly correlated features
def calculate_vif(df):
    vif_data = pd.DataFrame()
    vif_data["Feature"] = df.columns
    vif_data["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return vif_data.sort_values(by="VIF", ascending=False)

# Recalculate VIF
vif_data = calculate_vif(X)
print("\nUpdated VIF values:")
print(vif_data)

# Drop features with VIF > 10 iteratively
while vif_data["VIF"].max() > 10:
    feature_to_drop = vif_data.iloc[0]["Feature"]
    print(f"Dropping {feature_to_drop} due to high VIF...")
    X = X.drop(columns=[feature_to_drop])
    vif_data = calculate_vif(X)
    print("\nUpdated VIF values:")
    print(vif_data)

# Train the regression model after feature selection
X_const = sm.add_constant(X)  # Add intercept
ols_model = sm.OLS(y, X_const).fit()
print("\nFinal OLS Regression Results:\n", ols_model.summary())

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)



Updated VIF values:
                          Feature        VIF
21       UniqueGamesAttended_norm        inf
33               Tier_AB_Fraction        inf
31                Tier_D_Fraction        inf
30                Tier_C_Fraction        inf
29                Tier_B_Fraction        inf
28                Tier_A_Fraction        inf
27                   Tier_D_Count        inf
26                   Tier_C_Count        inf
25                   Tier_B_Count        inf
24                   Tier_A_Count        inf
22      BasketballPropensity_norm        inf
35         TierA_Weekday_Fraction        inf
20             GamesAttended_norm        inf
19     UniqueGamesAttended_capped        inf
18           GamesAttended_capped        inf
17     TotalTicketsPurchased_norm        inf
16   TotalTicketsPurchased_capped        inf
15  TotNumTicketsPurchased_capped        inf
14           DistanceToArena_norm        inf
13         DistanceToArena_capped        inf
12                  AvgSpend_norm 

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)



Updated VIF values:
                          Feature        VIF
20             GamesAttended_norm        inf
15  TotNumTicketsPurchased_capped        inf
30                Tier_D_Fraction        inf
29                Tier_C_Fraction        inf
28                Tier_B_Fraction        inf
27                Tier_A_Fraction        inf
26                   Tier_D_Count        inf
25                   Tier_C_Count        inf
24                   Tier_B_Count        inf
23                   Tier_A_Count        inf
21      BasketballPropensity_norm        inf
34         TierA_Weekday_Fraction        inf
18           GamesAttended_capped        inf
17     TotalTicketsPurchased_norm        inf
32               Tier_AB_Fraction        inf
16   TotalTicketsPurchased_capped        inf
14           DistanceToArena_norm        inf
13         DistanceToArena_capped        inf
12                  AvgSpend_norm        inf
11                AvgSpend_capped        inf
10          TotalTicketsPurchased 

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


...


Updated VIF values:
                          Feature           VIF
13  TotNumTicketsPurchased_capped           inf
30        TierAB_Weekday_Fraction           inf
20                   Tier_B_Count           inf
28         TierB_Weekday_Fraction           inf
27         TierA_Weekday_Fraction           inf
14   TotalTicketsPurchased_capped           inf
19                   Tier_A_Count           inf
10          TotalTicketsPurchased           inf
22                   Tier_D_Count           inf
21                   Tier_C_Count           inf
15     TotalTicketsPurchased_norm  1.200401e+09
17     UniqueGamesAttended_capped  2.056517e+01
31        TierCD_Weekday_Fraction  1.417518e+01
9             UniqueGamesAttended  1.250769e+01
26                WeekendFraction  1.047302e+01
29         TierD_Weekday_Fraction  8.450172e+00
16           GamesAttended_capped  4.715045e+00
6                   GamesAttended  4.414196e+00
0                          Season  3.393915e+00
4                  

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)



Updated VIF values:
                         Feature           VIF
19                  Tier_B_Count           inf
10         TotalTicketsPurchased           inf
29       TierAB_Weekday_Fraction           inf
27        TierB_Weekday_Fraction           inf
20                  Tier_C_Count           inf
26        TierA_Weekday_Fraction           inf
18                  Tier_A_Count           inf
21                  Tier_D_Count           inf
13  TotalTicketsPurchased_capped  1.594549e+09
14    TotalTicketsPurchased_norm  1.200401e+09
16    UniqueGamesAttended_capped  2.056517e+01
30       TierCD_Weekday_Fraction  1.417518e+01
9            UniqueGamesAttended  1.250769e+01
25               WeekendFraction  1.047302e+01
28        TierD_Weekday_Fraction  8.450172e+00
15          GamesAttended_capped  4.715045e+00
6                  GamesAttended  4.414196e+00
0                         Season  3.393915e+00
4                   GroupTickets  3.326987e+00
11               AvgSpend_capped  3.248

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)



Updated VIF values:
                         Feature           VIF
28       TierAB_Weekday_Fraction           inf
26        TierB_Weekday_Fraction           inf
25        TierA_Weekday_Fraction           inf
13  TotalTicketsPurchased_capped  1.594549e+09
14    TotalTicketsPurchased_norm  1.200401e+09
10         TotalTicketsPurchased  1.431309e+02
20                  Tier_D_Count  6.652375e+01
16    UniqueGamesAttended_capped  2.056517e+01
19                  Tier_C_Count  1.873003e+01
29       TierCD_Weekday_Fraction  1.417518e+01
9            UniqueGamesAttended  1.250769e+01
24               WeekendFraction  1.047302e+01
27        TierD_Weekday_Fraction  8.450172e+00
18                  Tier_A_Count  6.934771e+00
15          GamesAttended_capped  4.715045e+00
6                  GamesAttended  4.414196e+00
0                         Season  3.393915e+00
4                   GroupTickets  3.326987e+00
11               AvgSpend_capped  3.248160e+00
23               Tier_C_Fraction  3.212

In [59]:
import numpy as np

# Define a function to assign the FanSegment bonus
def get_fan_segment_bonus(segment):
    bonuses = {"B": 0.8, "G": 0.5, "E": 0.4}  # Assuming "E" is a duplicate, using 1.0 for "E"
    return bonuses.get(segment, 0)  # Default to 0 if segment is not in the list

# Compute the regression-based WeekendPlan_Score
def calculate_weekend_plan_score(row):
    score = (
        .60 * row["WeekendFraction"]
        + 0.10 * row["DistanceToArena"]
        + 0.10 * row["AvgSpend"]
        + 0.20 * get_fan_segment_bonus(row["FanSegment"])
    )
    return score


# Apply the function to compute scores
df["WeekendPlan_Score"] = df.apply(calculate_weekend_plan_score, axis=1)

min_score = df["WeekendPlan_Score"].min()
max_score = df["WeekendPlan_Score"].max()

df["WeekendPlan_Score"] = (df["WeekendPlan_Score"] - min_score) / (max_score - min_score)

# Display the relevant columns
columns_to_display = ["WeekendFraction", "FanSegment", "AvgSpend", "DistanceToArena", "WeekendPlan_Score"]
print(df[columns_to_display])

top_45 = df.sort_values(by="WeekendPlan_Score", ascending=False).head(45)
print(top_45[["WeekendFraction", "FanSegment", "AvgSpend", "DistanceToArena", "WeekendPlan_Score"]])

       WeekendFraction    FanSegment  AvgSpend  DistanceToArena  \
0                  0.0             F    467.00             12.0   
1                  0.0             A    116.00             47.0   
2                  0.0             B    107.00              6.0   
3                  0.0             C     27.00              3.0   
4                  0.0             A     14.00              4.0   
...                ...           ...       ...              ...   
42011              0.0             A    365.00             58.0   
42012              1.0             A      2.00             26.0   
42013              0.0             D      6.34              6.0   
42014              1.0  Limited Data     41.00              9.0   
42015              0.0             A     68.00              6.0   

       WeekendPlan_Score  
0               0.109037  
1               0.037104  
2               0.026087  
3               0.006829  
4               0.004097  
...                  ...  
42011 